# intro to ConvNets

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [12]:
model = Sequential()
# h = (h + 2*p - f)/s + 1
# w = (w + 2*p - f)/s + 1
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1))) #28x28 => 28-3+1 => 26x26
model.add(MaxPooling2D())                       #26x26 => 13x13
model.add(Conv2D(64, (3,3), activation='relu')) #13x13 => 11x11
model.add(MaxPooling2D())                       #11x11 => 5x5
model.add(Conv2D(64, (3,3), activation='relu')) #5x5 => 3x3
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                36928     
__________

In [8]:
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

In [9]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 4s 0us/step


In [11]:
x_train = x_train.astype('float32')/255
x_train = x_train.reshape((60000,28,28,1))
y_train = to_categorical(y_train)

x_test = x_test.astype('float32')/255
x_test = x_test.reshape((10000,28,28,1))
y_test = to_categorical(y_test)

In [13]:
history = model.fit(x_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 22s 363us/step - loss: 0.1684 - acc: 0.9476
Epoch 2/5
60000/60000 [==============================] - 21s 350us/step - loss: 0.0463 - acc: 0.9852
Epoch 3/5
60000/60000 [==============================] - 21s 349us/step - loss: 0.0328 - acc: 0.9901
Epoch 4/5
60000/60000 [==============================] - 21s 352us/step - loss: 0.0249 - acc: 0.9924
Epoch 5/5
60000/60000 [==============================] - 21s 350us/step - loss: 0.0202 - acc: 0.9938


In [14]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 150us/step


[0.023439019007971, 0.9918]

# dogs vs cats 

In [23]:
from keras.preprocessing.image import ImageDataGenerator

In [24]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [26]:
train_generator = train_datagen.flow_from_directory('./dogs-vs-cats/train/',
                                                    target_size=(150,150),class_mode='binary', batch_size=16)
val_generator = test_datagen.flow_from_directory('./dogs-vs-cats/validation/',
                                                    target_size=(150,150),class_mode='binary', batch_size=16)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [21]:
from keras import backend as K

In [35]:
K.clear_session()

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

In [36]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='binary_crossentropy',metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=125, epochs=30, validation_data=val_generator, validation_steps=62)